<a href="https://colab.research.google.com/github/JetteKA/BigData/blob/master/SSALrandom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import sklearn.metrics as metrics
from sklearn.cluster import KMeans
from keras.datasets import fashion_mnist
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

import keras
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.


In [0]:
n_classes = 10
num_centroids = 5

epochs = 15
batch_size = 50

img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)

In [0]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 2s 0us/step


In [0]:
def reshape_x_for_cnn(data, img_rows=img_rows, img_cols=img_cols):


  data = data.reshape(data.shape[0], img_rows, img_cols, 1)
  data = data.astype('float32')
  data /=255

  return data

def reshape_y_for_cnn(data, n_classes=n_classes):
  return keras.utils.to_categorical(data, n_classes)

In [0]:
x_test = reshape_x_for_cnn(x_test)

y_test = reshape_y_for_cnn(y_test)

In [0]:
def create_model():
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3),
                  activation='relu',
                  input_shape=input_shape))
  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(n_classes, activation='softmax'))

  model.compile(loss=keras.losses.categorical_crossentropy,
                optimizer=keras.optimizers.Adadelta(),
                metrics=['accuracy'])
  return model

In [0]:

train_acc = []
test_acc = []
train_loss = []
test_loss = []
num_labels = []

In [0]:
pseudo_label = []
actual_label = []

In [0]:
x_unlabelled = x_train[50:]
y_unlabelled = y_train[50:]

In [0]:
x_labelled = x_train[:50]
y_labelled = y_train[:50]

In [0]:
def train_and_label(model, x_labelled, y_labelled, x_unlabelled, y_unlabelled):


  x_label_cnn = reshape_x_for_cnn(x_labelled)


  y_label_cnn = reshape_y_for_cnn(y_labelled)

  model.fit(x_label_cnn, y_label_cnn,
            batch_size=batch_size,
            epochs=epochs,
            verbose=1)
  test_score = model.evaluate(x_test, y_test, verbose=0)
  # print('Test l0ccuracy:', test_score[1])
  test_loss.append(test_score[0])
  test_acc.append(test_score[1])

  train_score = model.evaluate(x_label_cnn, y_label_cnn, verbose=0)
  # print('Train loss:', train_score[0])
  # print('Train accuracy:', train_score[1])
  train_loss.append(train_score[0])
  train_acc.append(train_score[1])

  # print(x_labelled)

  x_unlabel_cnn = reshape_x_for_cnn(x_unlabelled)

  predictions = model.predict(x_unlabel_cnn)

  certainty = np.amax(predictions, axis=1)

  most_certain = certainty.argsort()[-50:]

  x_pseudo_label = []
  y_pseudo_label = []

  for i, index in enumerate(most_certain):

    x_pseudo_label.append(x_unlabelled[index])
    y_pseudo_label.append(np.argmax(predictions[index]))
    pseudo_label.append(np.argmax(predictions[index]))
    actual_label.append(y_unlabelled[index])

  most_uncertain = certainty.argsort()[:50]

  x_label_new = []
  y_label_new = []
  for i, index in enumerate(most_uncertain):

    x_label_new.append(x_unlabelled[index])
    y_label_new.append(y_unlabelled[index])

  y_labelled = np.array(y_labelled.tolist() + y_pseudo_label + y_label_new)
  x_labelled = np.array(x_labelled.tolist() + x_pseudo_label + x_label_new)

  x_unlabelled = np.delete(x_unlabelled, np.concatenate([most_certain, most_uncertain]), axis=0)
  y_unlabelled = np.delete(y_unlabelled, np.concatenate([most_certain, most_uncertain]))

  return model, x_labelled, y_labelled, x_unlabelled, y_unlabelled

In [0]:
  print('x label:', x_labelled.shape)
  print('y label:', y_labelled.shape)  
  print('x unlabel:', x_unlabelled.shape)
  print('y unlabel:', y_unlabelled.shape)

x label: (50, 28, 28)
y label: (50,)
x unlabel: (59950, 28, 28)
y unlabel: (59950,)


In [0]:
model = create_model()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [0]:
for i in range(100):
  model, x_labelled, y_labelled, x_unlabelled, y_unlabelled = train_and_label(model, x_labelled, y_labelled, x_unlabelled, y_unlabelled)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/15





50/50 [==============================] - 13s 257ms/step - loss: 2.2721 - acc: 0.1600
Epoch 2/15
50/50 [==============================] - 0s 135us/step - loss: 2.1392 - acc: 0.3000
Epoch 3/15
50/50 [==============================] - 0s 145us/step - loss: 1.9760 - acc: 0.4000
Epoch 4/15
50/50 [==============================] - 0s 130us/step - loss: 1.7953 - acc: 0.3800
Epoch 5/15
50/50 [==============================] - 0s 129us/step - loss: 1.5589 - acc: 0.5000
Epoch 6/15
50/50 [==============================] - 0s 143us/step - loss: 1.7866 - acc: 0.3600
Epoch 7/15
50/50 [==============================] - 0s 137us/step - loss: 1.6148 - acc: 0.4000
Epoch 8/15
50/50 [==============================] - 0s 130us/step - loss: 1.5048 - acc: 0.5200
Epoch 9/15
50/50 [==============================] - 0s 138us/step - loss: 1.1820 - acc: 0.6200
Epoch 10/15
50/50 [=======================

In [0]:
for i in range(100):
  model, x_labelled, y_labelled, x_unlabelled, y_unlabelled = train_and_label(model, x_labelled, y_labelled, x_unlabelled, y_unlabelled)

Epoch 1/15
10050/10050 [==============================] - 1s 114us/step - loss: 0.0518 - acc: 0.9873
Epoch 2/15
10050/10050 [==============================] - 1s 113us/step - loss: 0.0412 - acc: 0.9893
Epoch 3/15
10050/10050 [==============================] - 1s 115us/step - loss: 0.0315 - acc: 0.9919
Epoch 4/15
10050/10050 [==============================] - 1s 119us/step - loss: 0.0335 - acc: 0.9913
Epoch 5/15
10050/10050 [==============================] - 1s 114us/step - loss: 0.0419 - acc: 0.9905
Epoch 6/15
10050/10050 [==============================] - 1s 105us/step - loss: 0.0253 - acc: 0.9927
Epoch 7/15
10050/10050 [==============================] - 1s 107us/step - loss: 0.0300 - acc: 0.9916
Epoch 8/15
10050/10050 [==============================] - 1s 106us/step - loss: 0.0375 - acc: 0.9904
Epoch 9/15
10050/10050 [==============================] - 1s 105us/step - loss: 0.0304 - acc: 0.9924
Epoch 10/15
10050/10050 [==============================] - 1s 107us/step - loss: 0.0257 - a

In [0]:

# MyFile=open('drive/My Drive/BD/randomactivesemi_pseudo_labels.txt','w')

# for element in pseudo_label:
#      MyFile.write(str(element))
#      MyFile.write('\n')
# MyFile.close()

In [0]:

# MyFile=open('drive/My Drive/BD/randomactivesemi_actual_labels.txt','w')

# for element in actual_label:
#      MyFile.write(str(element))
#      MyFile.write('\n')
# MyFile.close()

In [0]:
num_labels = np.arange(50, 10050, 50)

In [0]:
plt.plot(num_labels, test_acc, label="Testset")
plt.plot(num_labels, train_acc, label="Trainset")
plt.xlabel('Number of labelled images')
plt.ylabel("Accuracy")
plt.axis(ymin=0, xmin=0, ymax=1.01)
plt.legend()
# plt.savefig("drive/My Drive/BD/randomactivesemi.png")

In [0]:

# MyFile=open('drive/My Drive/BD/randomactivesemi_50_acc.txt','w')

# for element in test_acc:
#      MyFile.write(str(element))
#      MyFile.write('\n')
# MyFile.close()


In [0]:

# MyFile=open('drive/My Drive/BD/randomactivesemi_num_labels.txt','w')

# for element in num_labels:
#      MyFile.write(str(element))
#      MyFile.write('\n')
# MyFile.close()

In [0]:

# MyFile=open('drive/My Drive/BD/randomactivesemi_50_acc_train.txt','w')

# for element in train_acc:
#      MyFile.write(str(element))
#      MyFile.write('\n')
# MyFile.close()